In [1]:
!pip install autogluon

In [62]:
import os
import pandas as pd
import random
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) 

In [63]:
train = pd.read_csv("C:/Users/user/Desktop/흡연여부/open/train.csv")
test = pd.read_csv("C:/Users/user/Desktop/흡연여부/open/test.csv")
sample_submission= pd.read_csv("C:/Users/user/Desktop/흡연여부/open/sample_submission.csv")


In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7000 non-null   object 
 1   나이        7000 non-null   int64  
 2   키(cm)     7000 non-null   int64  
 3   몸무게(kg)   7000 non-null   int64  
 4   BMI       7000 non-null   float64
 5   시력        7000 non-null   float64
 6   충치        7000 non-null   int64  
 7   공복 혈당     7000 non-null   int64  
 8   혈압        7000 non-null   int64  
 9   중성 지방     7000 non-null   int64  
 10  혈청 크레아티닌  7000 non-null   float64
 11  콜레스테롤     7000 non-null   int64  
 12  고밀도지단백    7000 non-null   int64  
 13  저밀도지단백    7000 non-null   int64  
 14  헤모글로빈     7000 non-null   float64
 15  요 단백      7000 non-null   int64  
 16  간 효소율     7000 non-null   float64
 17  label     7000 non-null   int64  
dtypes: float64(5), int64(12), object(1)
memory usage: 984.5+ KB


In [65]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        3000 non-null   object 
 1   나이        3000 non-null   int64  
 2   키(cm)     3000 non-null   int64  
 3   몸무게(kg)   3000 non-null   int64  
 4   BMI       3000 non-null   float64
 5   시력        3000 non-null   float64
 6   충치        3000 non-null   int64  
 7   공복 혈당     3000 non-null   int64  
 8   혈압        3000 non-null   int64  
 9   중성 지방     3000 non-null   int64  
 10  혈청 크레아티닌  3000 non-null   float64
 11  콜레스테롤     3000 non-null   int64  
 12  고밀도지단백    3000 non-null   int64  
 13  저밀도지단백    3000 non-null   int64  
 14  헤모글로빈     3000 non-null   float64
 15  요 단백      3000 non-null   int64  
 16  간 효소율     3000 non-null   float64
dtypes: float64(5), int64(11), object(1)
memory usage: 398.6+ KB


In [66]:
train = train.drop(['ID'], axis = 1)
test = test.drop('ID', axis = 1)

In [67]:
# 파생변수 =>혈청 크레아티닌값에 따라 정상/비정상 여부를 판단하여 1 또는 0으로 기록
train['혈청 크레아티닌_status'] = train['혈청 크레아티닌'].apply(lambda x: 1 if 0.8 <= x <= 1.7 else 0)


In [68]:
from sklearn.model_selection import train_test_split

#x_train, y_train 설정- 상관분석 고려한거
x_train = train.drop(['label'], axis=1)
y_train = train['label']

# 데이터셋을 학습용과 검증용으로 분할
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=0)


print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

(5600, 17) (5600,)
(1400, 17) (1400,)


In [69]:
# 학습데이터에 했던 전처리 과정 테스트데이터에도 진행
test['혈청 크레아티닌_status'] = test['혈청 크레아티닌'].apply(lambda x: 1 if 0.8 <= x <= 1.7 else 0)




In [77]:
train_data = TabularDataset(train)
test_data = TabularDataset(test)

label = 'label'
eval_metric = 'accuracy'
time_limit = 3600*2

In [78]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data,  presets='best_quality', time_limit=time_limit, num_stack_levels=3,num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels\ag-20230826_045546\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "AutogluonModels\ag-20230826_045546\"
AutoGluon Version:  0.8.2
Python Version:     3.8.8
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   88.12 GB / 237.11 GB (37.2%)
Train Data Rows:    7000
Train Data Columns: 17
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 =

In [79]:
predictor.leaderboard(silent = True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L5,0.759000,51.266534,3992.778888,0.029074,4.821012,5,True,50
1,WeightedEnsemble_L4,0.758571,35.703660,2935.906713,0.011065,3.591979,4,True,38
2,WeightedEnsemble_L3,0.756714,26.285770,2083.342374,0.030968,4.199351,3,True,26
3,CatBoost_BAG_L3,0.756714,31.509892,2609.678607,0.254165,154.623734,3,True,31
4,NeuralNetTorch_BAG_L3,0.756429,33.639285,2743.872811,2.383558,288.817937,3,True,36
5,NeuralNetTorch_BAG_L4,0.755714,46.649624,3685.370193,1.251787,151.769245,4,True,48
6,NeuralNetFastAI_BAG_L4,0.755143,47.252443,3670.781290,1.854606,137.180342,4,True,46
7,CatBoost_BAG_L4,0.754857,45.528789,3612.998828,0.130952,79.397880,4,True,43
8,NeuralNetFastAI_BAG_L3,0.754429,34.394150,2705.934392,3.138423,250.879519,3,True,34
9,LightGBMXT_BAG_L3,0.754286,32.782156,2528.718351,1.526429,73.663478,3,True,27


In [80]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data , model=model_to_use)

In [81]:
sample_submission['label'] = model_pred
sample_submission

,ID,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,0
...,...,...
2995,TEST_2995,0
2996,TEST_2996,0
2997,TEST_2997,0
2998,TEST_2998,1


In [82]:
sample_submission.to_csv("C:/Users/user/Desktop/흡연여부/open/submission_good.csv",index=False)

In [75]:
sample_submission.to_csv("C:/Users/user/Desktop/흡연여부/open/submission_best.csv",index=False)